# 读取数据

## 提取基差数据并进行差分

In [3]:
import new_factor as nf
import pandas as pd

myfactor = nf.NewFactor()
df_basis = myfactor.get_data('basis')
df_basis['T_diff'] = df_basis['T'].diff()
df_basis['TF_diff'] = df_basis['TF'].diff()
df_basis['TS_diff'] = df_basis['TS'].diff() 
df_basis['TL_diff'] = df_basis['TL'].diff()

df_basis.to_excel(f"factor/basis.xlsx")

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


## 读取因子数据

In [3]:
import data
data.run_data()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.
当前工作路径: d:\mycodelife\workshop\yinhe\daily_factor
类别: 利率利差
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: ./data/利率利差.xlsx

类别: 国债期货技术指标
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: ./data/国债期货技术指标.xlsx

类别: 资金面
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: ./data/资金面.xlsx

类别: 国债期货价差
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: ./data/国债期货价差.xlsx

类别: 股市
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: ./data/股市.xlsx

            CLOSE
2013-03-31    7.9
2013-06-30    7.6
2013-09-30    7.9
2013-12-31    7.7
2014-03-31    7.6
2014-06-30    7.7
2014-09-30    7.3
2014-12-31    7.4
2015-03-31    7.1
2015-06-30    7.1
2015-09-30    7.0


In [1]:
import factor 
factor.run_factor()

类别: 利率利差
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: factor/利率利差_20250924.xlsx

类别: 国债技术指标
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: factor/国债技术指标_20250924.xlsx

类别: 国债期货技术指标
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: factor/国债期货技术指标_20250924.xlsx

类别: 资金面
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: factor/资金面_20250924.xlsx

类别: 国债期货价差
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: factor/国债期货价差_20250924.xlsx

类别: 股市
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: factor/股市_20250924.xlsx

类别: 宏观
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: factor/宏观_20250924.xlsx

类别: 交易日期
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: factor/交易日期_20250924.xlsx



In [4]:
import os
import re
import pandas as pd

def merge_excels_by_first_col(folder_path, output_file="merged.xlsx"):
    # 排除：以8位日期开头、紧跟“-因子清单”的文件（不含扩展名匹配）
    pattern = re.compile(r"^\d{8}-因子清单")

    files = [
        f for f in os.listdir(folder_path)
        if f.endswith(".xlsx")
        and not f.startswith("~")
        and not pattern.match(os.path.splitext(f)[0])
    ]
    if not files:
        print("未找到任何可用的 .xlsx 文件")
        return

    dfs = []
    for f in sorted(files):
        path = os.path.join(folder_path, f)
        try:
            # 第一行是列名，第一列作为索引（合并键）
            df = pd.read_excel(path, header=0, index_col=0)
        except Exception as e:
            print(f"读取失败：{path}，原因：{e}")
            continue

        # 规范化索引（第一列），避免空白/类型不一致
        df.index = df.index.astype(str).str.strip()

        # 列名加前缀（保留原始列名，避免冲突）
        stem = os.path.splitext(f)[0]
        df.columns = [f"{stem}__{col}" for col in df.columns]

        dfs.append(df)

    if not dfs:
        print("没有成功读取的表格")
        return

    # 以索引（第一列）按“交集”对齐拼接
    merged = pd.concat(dfs, axis=1, join="inner")

    # 索引转为普通列，并命名为 "date"
    merged = merged.reset_index().rename(columns={"index": "date"})

    # 导出
    merged.to_excel(output_file, index=False, header=True)
    print(f"合并完成，结果已保存到 {output_file}")
    print(f"合并后形状：{merged.shape}（行=共有键数量，列=1+各文件数据列总数）")

# 用法
merge_excels_by_first_col(r"factor", output_file="merged.xlsx")


合并完成，结果已保存到 merged.xlsx
合并后形状：(2122, 771)（行=共有键数量，列=1+各文件数据列总数）


In [9]:
import numpy as np
import pandas as pd

df_raw = pd.read_excel('merged.xlsx')
df_raw.set_index('date', inplace=True)

# 涨跌标签：下一期比当前期涨为 1，跌为 -1，不变为 0
df_raw['label'] = (df_raw['basis__TS'].shift(-1) - df_raw['basis__TS']).apply(
    lambda x: 1 if x > 0 else -1 if x < 0 else 0
)
# 删除最后一行（因为没有未来数据做标签）
df_clean = df_raw.iloc[:-1]

In [10]:
# 假设 df_clean 已经准备好，索引为时间或顺序

label_col = "label"
split_point = int(len(df_clean) * 0.8)

# 按顺序切分
df_train = df_clean.iloc[:split_point]
df_test  = df_clean.iloc[split_point:]

# 拆分特征和标签
X_train, y_train = df_train.drop(columns=[label_col]), df_train[label_col]
X_test,  y_test  = df_test.drop(columns=[label_col]),  df_test[label_col]

train_data = pd.concat([X_train,y_train],axis=1)
test_data = pd.concat([X_test, y_test],axis = 1)

In [14]:
from autogluon.tabular import TabularDataset, TabularPredictor
predictor = TabularPredictor(label=label_col).fit(train_data)

No path specified. Models will be saved in: "AutogluonModels\ag-20250926_041940"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          16
Memory Avail:       6.51 GB / 15.73 GB (41.4%)
Disk Space Avail:   79.16 GB / 274.71 GB (28.8%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory and 32+ GB GPU memory.
	presets='best'    : Maximize accu

In [15]:
predictor.evaluate(test_data)

{'accuracy': 0.5247058823529411,
 'balanced_accuracy': np.float64(0.5338725424932321),
 'mcc': 0.07409372538778082}

In [13]:
predictor.path

'd:\\mycodelife\\workshop\\yinhe\\daily_factor\\AutogluonModels\\ag-20250926_024054'

In [12]:
predictor = TabularPredictor.load("d:\\mycodelife\\workshop\\yinhe\\daily_factor\\AutogluonModels\\best_T")

# 永续债

In [9]:
import data
import new_factor as nf
import numpy as np
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
import factor
import re
import os, sys
from datetime import datetime

In [3]:
import data
import new_factor as nf
import numpy as np
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
import factor
import re
import os, sys
from datetime import datetime

# 把脚本所在目录设为当前工作目录
# os.chdir(os.path.dirname(os.path.abspath(__file__)))

def merge_excels_by_first_col(folder_path, output_file="merged.xlsx"):
    # 排除：以8位日期开头、紧跟“-因子清单”的文件（不含扩展名匹配）
    pattern = re.compile(r"^\d{8}-因子清单")
    files = [
        f for f in os.listdir(folder_path)
        if f.endswith(".xlsx")
        and not f.startswith("~")
        and not pattern.match(os.path.splitext(f)[0])
    ]
    if not files:
        print("未找到任何可用的 .xlsx 文件")
        return
    dfs = []
    for f in sorted(files):
        path = os.path.join(folder_path, f)
        try:
            # 第一行是列名，第一列作为索引（合并键）
            df = pd.read_excel(path, header=0, index_col=0)
        except Exception as e:
            print(f"读取失败：{path}，原因：{e}")
            continue
        # 规范化索引（第一列），避免空白/类型不一致
        df.index = df.index.astype(str).str.strip()
        # 列名加前缀（保留原始列名，避免冲突）
        stem = os.path.splitext(f)[0]
        df.columns = [f"{stem}__{col}" for col in df.columns]
        dfs.append(df)
    if not dfs:
        print("没有成功读取的表格")
        return
    # 以索引（第一列）按“交集”对齐拼接
    merged = pd.concat(dfs, axis=1, join="inner")
    # 索引转为普通列，并命名为 "date"
    merged = merged.reset_index().rename(columns={"index": "date"})
    # 导出
    merged.to_excel(output_file, index=False, header=True)
    return merged
data.run_data()
factor.run_factor()
myfactor = nf.NewFactor()
myfactor.get_data('FR007').to_excel('factor/FR007.xlsx')
df_raw = merge_excels_by_first_col('factor', 'merged_FR007.xlsx')

当前工作路径: d:\mycodelife\workshop\yinhe\daily_factor
类别: 利率利差
保存并更新数据成功
时间: 20170101 - 20251016
当前存在数据时间范围: 20170101 - 20251016
保存路径: ./data/利率利差.xlsx

类别: 国债期货技术指标
保存并更新数据成功
时间: 20170101 - 20251016
当前存在数据时间范围: 20170101 - 20251016
保存路径: ./data/国债期货技术指标.xlsx

类别: 资金面
保存并更新数据成功
时间: 20170101 - 20251016
当前存在数据时间范围: 20170101 - 20251016
保存路径: ./data/资金面.xlsx

类别: 国债期货价差
保存并更新数据成功
时间: 20170101 - 20251016
当前存在数据时间范围: 20170101 - 20251016
保存路径: ./data/国债期货价差.xlsx

类别: 股市
保存并更新数据成功
时间: 20170101 - 20251016
当前存在数据时间范围: 20170101 - 20251016
保存路径: ./data/股市.xlsx

            CLOSE
2013-03-31    7.9
2013-06-30    7.6
2013-09-30    7.9
2013-12-31    7.7
2014-03-31    7.6
2014-06-30    7.7
2014-09-30    7.3
2014-12-31    7.4
2015-03-31    7.1
2015-06-30    7.1
2015-09-30    7.0
2015-12-31    6.9
2016-03-31    6.8
2016-06-30    6.7
2016-09-30    6.7
2016-12-31    6.8
2017-03-31    7.0
2017-06-30    7.0
2017-09-30    6.9
2017-12-31    6.8
2018-03-31    6.9
2018-06-30    6.9
2018-09-30    6.7
2018-12-31    6.5

In [10]:
import pandas as pd

df_raw = pd.read_excel('merged_perpetual.xlsx')
df_raw.set_index('date', inplace=True)

# 先删除目标列为空的行
df_raw = df_raw.dropna(subset=['perpetual__永续债_5Y'])

# 涨跌标签：下一期比当前期涨为 1，跌为 -1，不变为 0
df_raw['label'] = (df_raw['perpetual__永续债_5Y'].shift(-1) - df_raw['perpetual__永续债_5Y']).apply(
    lambda x: 1 if x > 0 else (-1 if x < 0 else 0)
)

# 若为 0，则继承前一个标签
df_raw['label'] = df_raw['label'].replace(0, method='ffill')

# 删除最后一行（因为没有未来数据做标签）
df_clean = df_raw.iloc[:-1]

# 假设 df_clean 已经准备好，索引为时间或顺序
label_col = "label"
split_point = int(len(df_clean) * 0.8)

# 按顺序切分
df_train = df_clean.iloc[:split_point]
df_test  = df_clean.iloc[split_point:]

# 拆分特征和标签
X_train, y_train = df_train.drop(columns=[label_col]), df_train[label_col]
X_test,  y_test  = df_test.drop(columns=[label_col]),  df_test[label_col]

train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

C:\Users\86198\AppData\Local\Temp\ipykernel_3460\1388561248.py:15: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df_raw['label'] = df_raw['label'].replace(0, method='ffill')


In [12]:
df_clean['label'].value_counts()

label
-1    573
 1    465
Name: count, dtype: int64

## 训练

In [13]:
from autogluon.tabular import TabularDataset, TabularPredictor
predictor = TabularPredictor(label=label_col).fit(train_data,presets='best')

No path specified. Models will be saved in: "AutogluonModels\ag-20251019_021749"
Preset alias specified: 'best' maps to 'best_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          16
Memory Avail:       4.04 GB / 15.73 GB (25.7%)
Disk Space Avail:   37.74 GB / 274.71 GB (13.7%)
Presets specified: ['best']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify t

In [14]:
# 五年的
predictor.evaluate(test_data,auxiliary_metrics=True)

{'accuracy': 0.5961538461538461,
 'balanced_accuracy': np.float64(0.596372721384288),
 'mcc': 0.1926741413908403,
 'roc_auc': np.float64(0.6153419080225779),
 'f1': 0.6,
 'precision': 0.6116504854368932,
 'recall': 0.5887850467289719}

In [10]:
# 七年的
predictor.evaluate(test_data,auxiliary_metrics=True)

{'accuracy': 0.6394230769230769,
 'balanced_accuracy': np.float64(0.6397460847002132),
 'mcc': 0.27916897475798874,
 'roc_auc': np.float64(0.7007691594847558),
 'f1': 0.647887323943662,
 'precision': 0.6634615384615384,
 'recall': 0.6330275229357798}

In [9]:
# 七年的
predictor.path

'd:\\mycodelife\\workshop\\yinhe\\daily_factor\\AutogluonModels\\ag-20251016_070855'

In [3]:
import pandas as pd
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor.load("d:\\mycodelife\\workshop\\yinhe\\daily_factor\\AutogluonModels\\ag-20251016_070855")
fi = predictor.feature_importance(data=test_data)
fi.to_excel('永续债七年.xlsx')

These features in provided data are not utilized by the predictor and will be ignored: ['20251016-因子__是否交易日', '20251016-因子交易日__是否交易日', '20251016-因子交易日__月', '20251016-因子交易日__日', '20251016-因子交易日__是否15号', '20251016-因子交易日__是否20号', '20251016-因子交易日__星期', '20251016-因子交易日__国债_10Y-国开债_10Y', '20251016-因子交易日__国债_10Y-企业债AAA+_10Y', '20251016-因子交易日__国债_10Y-中短期票据AAA+_10Y', '20251016-因子交易日__国债_10Y-国债_1Y', '20251016-因子交易日__国债_10Y-国债_3Y', '20251016-因子交易日__国债_10Y-国债_5Y', '20251016-因子交易日__国债_10Y-国债_7Y', '20251016-因子交易日__国债_30Y-国债_10Y', '20251016-因子交易日__国开债_10Y-国债_10Y', '20251016-因子交易日__国开债_5Y-国债_5Y', '20251016-因子交易日__国开债_1Y-国债_1Y', '20251016-因子交易日__中短期票据AAA+_1Y-国开债1Y', '20251016-因子交易日__中短期票据AAA+_3Y-国开债3Y', '20251016-因子交易日__中短期票据AAA+_5Y-国开债5Y', '20251016-因子交易日__中短期票据AAA+_7Y-国开债7Y', '20251016-因子交易日__中短期票据AAA+_10Y-国开债10Y', '20251016-因子交易日__中短期票据AAA+_30Y-国开债30Y', '20251016-因子交易日__国债_10Y-同业存单AAA_1Y', '20251016-因子交易日__国开债_10Y-国开债_5Y', '20251016-因子交易日__国开债_10Y-国开债_1Y', '20251016-因子交易日__R007-R001', '20251016-因子交易

In [4]:
used_features = predictor.feature_metadata.get_features()
print(used_features)

['20251016-因子__月', '20251016-因子__日', '20251016-因子__是否15号', '20251016-因子__是否20号', '20251016-因子__星期', '20251016-因子__国债_10Y-国开债_10Y', '20251016-因子__国债_10Y-企业债AAA+_10Y', '20251016-因子__国债_10Y-中短期票据AAA+_10Y', '20251016-因子__国债_10Y-国债_1Y', '20251016-因子__国债_10Y-国债_3Y', '20251016-因子__国债_10Y-国债_5Y', '20251016-因子__国债_10Y-国债_7Y', '20251016-因子__国债_30Y-国债_10Y', '20251016-因子__国开债_10Y-国债_10Y', '20251016-因子__国开债_5Y-国债_5Y', '20251016-因子__国开债_1Y-国债_1Y', '20251016-因子__中短期票据AAA+_1Y-国开债1Y', '20251016-因子__中短期票据AAA+_3Y-国开债3Y', '20251016-因子__中短期票据AAA+_5Y-国开债5Y', '20251016-因子__中短期票据AAA+_7Y-国开债7Y', '20251016-因子__中短期票据AAA+_10Y-国开债10Y', '20251016-因子__中短期票据AAA+_30Y-国开债30Y', '20251016-因子__国债_10Y-同业存单AAA_1Y', '20251016-因子__国开债_10Y-国开债_5Y', '20251016-因子__国开债_10Y-国开债_1Y', '20251016-因子__R007-R001', '20251016-因子__活跃券_10Y-活跃券_3Y', '20251016-因子__活跃券_10Y-活跃券_5Y', '20251016-因子__活跃券_10Y-活跃券_7Y', '20251016-因子__活跃券_10Y-国债_10Y', '20251016-因子__FR007_IRS_5Y-FR007_IRS_1Y', '20251016-因子__FR007_IRS_1Y-FR007_IRS_9M', '20251016-因子__国债_5

In [16]:
import pandas as pd
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor.load('d:\\mycodelife\\workshop\\yinhe\\daily_factor\\AutogluonModels\\ag-20251015_065659')
fi = predictor.feature_importance(data=test_data)
fi.to_excel('永续债五年.xlsx')

These features in provided data are not utilized by the predictor and will be ignored: ['20251016-因子__是否交易日', '20251016-因子__月', '20251016-因子__日', '20251016-因子__是否15号', '20251016-因子__是否20号', '20251016-因子__星期', '20251016-因子__国债_10Y-国开债_10Y', '20251016-因子__国债_10Y-企业债AAA+_10Y', '20251016-因子__国债_10Y-中短期票据AAA+_10Y', '20251016-因子__国债_10Y-国债_1Y', '20251016-因子__国债_10Y-国债_3Y', '20251016-因子__国债_10Y-国债_5Y', '20251016-因子__国债_10Y-国债_7Y', '20251016-因子__国债_30Y-国债_10Y', '20251016-因子__国开债_10Y-国债_10Y', '20251016-因子__国开债_5Y-国债_5Y', '20251016-因子__国开债_1Y-国债_1Y', '20251016-因子__中短期票据AAA+_1Y-国开债1Y', '20251016-因子__中短期票据AAA+_3Y-国开债3Y', '20251016-因子__中短期票据AAA+_5Y-国开债5Y', '20251016-因子__中短期票据AAA+_7Y-国开债7Y', '20251016-因子__中短期票据AAA+_10Y-国开债10Y', '20251016-因子__中短期票据AAA+_30Y-国开债30Y', '20251016-因子__国债_10Y-同业存单AAA_1Y', '20251016-因子__国开债_10Y-国开债_5Y', '20251016-因子__国开债_10Y-国开债_1Y', '20251016-因子__R007-R001', '20251016-因子__活跃券_10Y-活跃券_3Y', '20251016-因子__活跃券_10Y-活跃券_5Y', '20251016-因子__活跃券_10Y-活跃券_7Y', '20251016-因子__活跃券_10Y-国债

KeyError: "253 required columns are missing from the provided dataset to transform using AutoMLPipelineFeatureGenerator. 253 missing columns: ['20251015-因子__月', '20251015-因子__日', '20251015-因子__是否15号', '20251015-因子__是否20号', '20251015-因子__星期', '20251015-因子__国债_10Y-国开债_10Y', '20251015-因子__国债_10Y-企业债AAA+_10Y', '20251015-因子__国债_10Y-中短期票据AAA+_10Y', '20251015-因子__国债_10Y-国债_1Y', '20251015-因子__国债_10Y-国债_3Y', '20251015-因子__国债_10Y-国债_5Y', '20251015-因子__国债_10Y-国债_7Y', '20251015-因子__国债_30Y-国债_10Y', '20251015-因子__国开债_10Y-国债_10Y', '20251015-因子__国开债_5Y-国债_5Y', '20251015-因子__国开债_1Y-国债_1Y', '20251015-因子__中短期票据AAA+_1Y-国开债1Y', '20251015-因子__中短期票据AAA+_3Y-国开债3Y', '20251015-因子__中短期票据AAA+_5Y-国开债5Y', '20251015-因子__中短期票据AAA+_7Y-国开债7Y', '20251015-因子__中短期票据AAA+_10Y-国开债10Y', '20251015-因子__中短期票据AAA+_30Y-国开债30Y', '20251015-因子__国债_10Y-同业存单AAA_1Y', '20251015-因子__国开债_10Y-国开债_5Y', '20251015-因子__国开债_10Y-国开债_1Y', '20251015-因子__R007-R001', '20251015-因子__活跃券_10Y-活跃券_3Y', '20251015-因子__活跃券_10Y-活跃券_5Y', '20251015-因子__活跃券_10Y-

In [15]:
# 五年的
predictor.path

'd:\\mycodelife\\workshop\\yinhe\\daily_factor\\AutogluonModels\\ag-20251019_021749'

In [17]:
# 查看模型实际使用的特征名
used_cols = predictor.feature_metadata.get_features()

# 显示当前 test_data 的列名（排除 label）
test_cols = [col for col in test_data.columns if col != 'label']

# 比对差异
print("缺失特征：", set(used_cols) - set(test_cols))
print("多余特征：", set(test_cols) - set(used_cols))


缺失特征： {'20251015-因子__国债期货_5Y_wr_10', '20251015-因子__国债期货_5Y_atr', '20251015-因子__国债_1Y_macd', '20251015-因子__国债_30Y_收益率变动_30D', '20251015-因子__国债_30Y_macd', '20251015-因子__国债期货_10Y_wr_20', '20251015-因子__国债期货_2Y_wr_20', '20251015-因子__国债期货_2Y_wr_5', '20251015-因子__FR007_5日标准差', '20251015-因子__国债_10Y-同业存单AAA_1Y', '20251015-因子__国债期货_5Y_klow2', '20251015-因子__国债期货_5Y_trima_5', '20251015-因子__SHIBOR_1Y-FR007_IRS_1Y', '20251015-因子__国债期货_5Y_cmo_10', '20251015-因子__国开债_10Y-国债_10Y', '20251015-因子__国债期货_5Y_adi', '20251015-因子__国债期货_5Y_low2high', '20251015-因子__活跃券_10Y-活跃券_7Y', '20251015-因子__是否15号', '20251015-因子__国债期货_10Y_roc_5', '20251015-因子__国债期货_10Y_roc_10', '20251015-因子__国债期货_5Y_open', '20251015-因子__国债期货_30Y_cmo_20', '20251015-因子__国债期货_5Y_momentum', '20251015-因子__国债期货_30Y_cmo_5', '20251015-因子__国债期货_2Y_vwap', '20251015-因子__国债期货_30Y_kdj_5', '20251015-因子__SHIROR_3M_5日移动平均', '20251015-因子__FR001_5日标准差', '20251015-因子__国债_7Y_收益率变动_5D', '20251015-因子__国债期货_10Y_obv', '20251015-因子__国债期货_10Y_kup2', '20251015-因子__美元指数'

# 资本债

In [1]:
import data
import new_factor as nf
import numpy as np
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
import factor
import re
import os, sys
from datetime import datetime

d:\conda\envs\pavane\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
myfactor.get_data('tier2').to_excel('factor/tier2.xlsx')
df_raw = merge_excels_by_first_col('factor', 'merged.xlsx')

In [3]:
df_raw = pd.read_excel('merged_tier2.xlsx')
df_raw.set_index('date', inplace=True)

# 涨跌标签：下一期比当前期涨为 1，跌为 -1，不变为 0
df_raw['label'] = (df_raw['tier2__二级资本债_5Y'].shift(-1) - df_raw['tier2__二级资本债_5Y']).apply(
    lambda x: 1 if x > 0 else -1
)
# 删除最后一行（因为没有未来数据做标签）
df_clean = df_raw.iloc[:-1]
# 假设 df_clean 已经准备好，索引为时间或顺序

label_col = "label"
split_point = int(len(df_clean) * 0.8)

# 按顺序切分
df_train = df_clean.iloc[:split_point]
df_test  = df_clean.iloc[split_point:]

# 拆分特征和标签
X_train, y_train = df_train.drop(columns=[label_col]), df_train[label_col]
X_test,  y_test  = df_test.drop(columns=[label_col]),  df_test[label_col]

train_data = pd.concat([X_train,y_train],axis=1)
test_data = pd.concat([X_test, y_test],axis = 1)

## 训练

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
predictor = TabularPredictor(label=label_col).fit(train_data)

No path specified. Models will be saved in: "AutogluonModels\ag-20251015_014109"
Preset alias specified: 'best' maps to 'best_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          16
Memory Avail:       3.33 GB / 15.73 GB (21.1%)
Disk Space Avail:   69.09 GB / 274.71 GB (25.2%)
Presets specified: ['best']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify t

In [18]:
predictor.evaluate(test_data,auxiliary_metrics=True)

{'accuracy': 0.5023474178403756,
 'balanced_accuracy': np.float64(0.40138952511833875),
 'mcc': 0.16404065590559166}

In [20]:
from sklearn.metrics import classification_report, roc_auc_score

y_true = test_data["label"]
y_pred = predictor.predict(test_data)
y_prob = predictor.predict_proba(test_data)[1]  # 概率

print(classification_report(y_true, y_pred))
print("AUC:", roc_auc_score(y_true, y_prob))


              precision    recall  f1-score   support

          -1       0.66      0.53      0.59       236
           0       0.01      0.20      0.02         5
           1       0.56      0.47      0.51       185

    accuracy                           0.50       426
   macro avg       0.41      0.40      0.37       426
weighted avg       0.61      0.50      0.55       426



ValueError: multi_class must be in ('ovo', 'ovr')

In [6]:
predictor.path

'd:\\mycodelife\\workshop\\yinhe\\daily_factor\\AutogluonModels\\ag-20251015_014109'

# FR007

In [ ]:
import data
import new_factor as nf
import numpy as np
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
import factor
import re
import os, sys
from datetime import datetime

In [ ]:
myfactor.get_data('tier2').to_excel('factor/tier2.xlsx')
df_raw = merge_excels_by_first_col('factor', 'merged.xlsx')

In [ ]:
df_raw = pd.read_excel('merged.xlsx')
df_raw.set_index('date', inplace=True)

# 涨跌标签：下一期比当前期涨为 1，跌为 -1，不变为 0
df_raw['label'] = (df_raw['tier2__二级资本债_5Y'].shift(-1) - df_raw['tier2__二级资本债_5Y']).apply(
    lambda x: 1 if x > 0 else -1 if x < 0 else 0
)
# 删除最后一行（因为没有未来数据做标签）
df_clean = df_raw.iloc[:-1]
# 假设 df_clean 已经准备好，索引为时间或顺序

label_col = "label"
split_point = int(len(df_clean) * 0.8)

# 按顺序切分
df_train = df_clean.iloc[:split_point]
df_test  = df_clean.iloc[split_point:]

# 拆分特征和标签
X_train, y_train = df_train.drop(columns=[label_col]), df_train[label_col]
X_test,  y_test  = df_test.drop(columns=[label_col]),  df_test[label_col]

train_data = pd.concat([X_train,y_train],axis=1)
test_data = pd.concat([X_test, y_test],axis = 1)

# 国债

In [1]:
import data
import new_factor as nf
import numpy as np
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
import factor
import re
import os, sys
from datetime import datetime

# 把脚本所在目录设为当前工作目录
# os.chdir(os.path.dirname(os.path.abspath(__file__)))

def merge_excels_by_first_col(folder_path, output_file="merged.xlsx"):
    # 排除：以8位日期开头、紧跟“-因子清单”的文件（不含扩展名匹配）
    pattern = re.compile(r"^\d{8}-因子清单")
    files = [
        f for f in os.listdir(folder_path)
        if f.endswith(".xlsx")
        and not f.startswith("~")
        and not pattern.match(os.path.splitext(f)[0])
    ]
    if not files:
        print("未找到任何可用的 .xlsx 文件")
        return
    dfs = []
    for f in sorted(files):
        path = os.path.join(folder_path, f)
        try:
            # 第一行是列名，第一列作为索引（合并键）
            df = pd.read_excel(path, header=0, index_col=0)
        except Exception as e:
            print(f"读取失败：{path}，原因：{e}")
            continue
        # 规范化索引（第一列），避免空白/类型不一致
        df.index = df.index.astype(str).str.strip()
        # 列名加前缀（保留原始列名，避免冲突）
        stem = os.path.splitext(f)[0]
        df.columns = [f"{stem}__{col}" for col in df.columns]
        dfs.append(df)
    if not dfs:
        print("没有成功读取的表格")
        return
    # 以索引（第一列）按“交集”对齐拼接
    merged = pd.concat(dfs, axis=1, join="inner")
    # 索引转为普通列，并命名为 "date"
    merged = merged.reset_index().rename(columns={"index": "date"})
    # 导出
    merged.to_excel(output_file, index=False, header=True)
    return merged
data.run_data()
factor.run_factor()
myfactor = nf.NewFactor()
myfactor.get_data('national').to_excel('factor/national.xlsx')
df_raw = merge_excels_by_first_col('factor', 'merged_national.xlsx')

d:\conda\envs\pavane\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.
当前工作路径: d:\mycodelife\workshop\yinhe\daily_factor
类别: 利率利差
保存并更新数据成功
时间: 20170101 - 20251020
当前存在数据时间范围: 20170101 - 20251020
保存路径: ./data/利率利差.xlsx

类别: 国债期货技术指标
保存并更新数据成功
时间: 20170101 - 20251020
当前存在数据时间范围: 20170101 - 20251020
保存路径: ./data/国债期货技术指标.xlsx

类别: 资金面
保存并更新数据成功
时间: 20170101 - 20251020
当前存在数据时间范围: 20170101 - 20251020
保存路径: ./data/资金面.xlsx

类别: 国债期货价差
保存并更新数据成功
时间: 20170101 - 20251020
当前存在数据时间范围: 20170101 - 20251020
保存路径: ./data/国债期货价差.xlsx

类别: 股市
保存并更新数据成功
时间: 20170101 - 20251020
当前存在数据时间范围: 20170101 - 20251020
保存路径: ./data/股市.xlsx

            CLOSE
2013-03-31    7.9
2013-06-30    7.6
2013-09-30    7.9
2013-12-31    7.7
2014-03-31    7.6
2014-06-30    7.7
2014-09-30    7.3
2014-12-31    7.4
2015-03-31    7.1
2015-06-30    7.1
2015-09-30    7.0


In [3]:
import pandas as pd

df_raw = pd.read_excel('merged_national.xlsx')
df_raw.set_index('date', inplace=True)

# 先删除目标列为空的行
df_raw = df_raw.dropna(subset=['national__国债_10Y'])

# 涨跌标签：下一期比当前期涨为 1，跌为 -1，不变为 0
df_raw['label'] = (df_raw['national__国债_10Y'].shift(-1) - df_raw['national__国债_10Y']).apply(
    lambda x: -1 if x > 0 else (1 if x < 0 else 0)
)

# 若为 0，则继承前一个标签
df_raw['label'] = df_raw['label'].replace(0, method='ffill')

# 删除最后一行（因为没有未来数据做标签）
df_clean = df_raw.iloc[:-1]

# 假设 df_clean 已经准备好，索引为时间或顺序
label_col = "label"
split_point = int(len(df_clean) * 0.8)

# 按顺序切分
df_train = df_clean.iloc[:split_point]
df_test  = df_clean.iloc[split_point:]

# 拆分特征和标签
X_train, y_train = df_train.drop(columns=[label_col]), df_train[label_col]
X_test,  y_test  = df_test.drop(columns=[label_col]),  df_test[label_col]

train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

C:\Users\86198\AppData\Local\Temp\ipykernel_27052\1040909832.py:15: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df_raw['label'] = df_raw['label'].replace(0, method='ffill')


In [4]:
from autogluon.tabular import TabularDataset, TabularPredictor
predictor = TabularPredictor(label=label_col).fit(train_data,presets='best')

No path specified. Models will be saved in: "AutogluonModels\ag-20251020_031328"
Preset alias specified: 'best' maps to 'best_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          16
Memory Avail:       5.90 GB / 15.73 GB (37.5%)
Disk Space Avail:   37.05 GB / 274.71 GB (13.5%)
Presets specified: ['best']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify t

## 信号与重要性

In [5]:
predictor.evaluate(test_data,auxiliary_metrics=True)

{'accuracy': 0.6978922716627635,
 'balanced_accuracy': np.float64(0.6819075273620728),
 'mcc': 0.3765182856252967,
 'roc_auc': np.float64(0.7527585436676345),
 'f1': 0.7504835589941973,
 'precision': 0.7054545454545454,
 'recall': 0.8016528925619835}

In [6]:
predictor.path

'd:\\mycodelife\\workshop\\yinhe\\daily_factor\\AutogluonModels\\ag-20251020_031328'

In [7]:
from autogluon.tabular import TabularPredictor
import pandas as pd

# 加载模型并预测
# predictor = TabularPredictor.load("/root/AutogluonModels/ag-20251017_002741")
predictions = predictor.predict(test_data)

# 构建结果 DataFrame
df_result = test_data.reset_index()[['date', 'national__国债_10Y', 'label']].copy()
df_result['predictions'] = predictions.values

# 所有数值取负，但国债修复了所以不用
# df_result['label'] = -df_result['label']
# df_result['predictions'] = -df_result['predictions']

# 保存结果
output_file = 'prediction_results_国债十年.xlsx'
df_result.to_excel(output_file, index=False)

print(f"预测结果已保存到 {output_file}")

预测结果已保存到 prediction_results_国债十年.xlsx


In [8]:
import pandas as pd
from autogluon.tabular import TabularPredictor
# predictor = TabularPredictor.load("/root/AutogluonModels/ag-20251016_045333")
fi = predictor.feature_importance(data=test_data)
fi.to_excel('国债十年.xlsx')

These features in provided data are not utilized by the predictor and will be ignored: ['20251020-因子__是否交易日', '20251020-因子交易日__是否交易日', '20251020-因子交易日__月', '20251020-因子交易日__日', '20251020-因子交易日__是否15号', '20251020-因子交易日__是否20号', '20251020-因子交易日__星期', '20251020-因子交易日__国债_10Y-国开债_10Y', '20251020-因子交易日__国债_10Y-企业债AAA+_10Y', '20251020-因子交易日__国债_10Y-中短期票据AAA+_10Y', '20251020-因子交易日__国债_10Y-国债_1Y', '20251020-因子交易日__国债_10Y-国债_3Y', '20251020-因子交易日__国债_10Y-国债_5Y', '20251020-因子交易日__国债_10Y-国债_7Y', '20251020-因子交易日__国债_30Y-国债_10Y', '20251020-因子交易日__国开债_10Y-国债_10Y', '20251020-因子交易日__国开债_5Y-国债_5Y', '20251020-因子交易日__国开债_1Y-国债_1Y', '20251020-因子交易日__中短期票据AAA+_1Y-国开债1Y', '20251020-因子交易日__中短期票据AAA+_3Y-国开债3Y', '20251020-因子交易日__中短期票据AAA+_5Y-国开债5Y', '20251020-因子交易日__中短期票据AAA+_7Y-国开债7Y', '20251020-因子交易日__中短期票据AAA+_10Y-国开债10Y', '20251020-因子交易日__中短期票据AAA+_30Y-国开债30Y', '20251020-因子交易日__国债_10Y-同业存单AAA_1Y', '20251020-因子交易日__国开债_10Y-国开债_5Y', '20251020-因子交易日__国开债_10Y-国开债_1Y', '20251020-因子交易日__R007-R001', '20251020-因子交易

# 国开债

In [10]:
import data
import new_factor as nf
import numpy as np
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
import factor
import re
import os, sys
from datetime import datetime

# 把脚本所在目录设为当前工作目录
# os.chdir(os.path.dirname(os.path.abspath(__file__)))

def merge_excels_by_first_col(folder_path, output_file="merged.xlsx"):
    # 排除：以8位日期开头、紧跟“-因子清单”的文件（不含扩展名匹配）
    pattern = re.compile(r"^\d{8}-因子清单")
    files = [
        f for f in os.listdir(folder_path)
        if f.endswith(".xlsx")
        and not f.startswith("~")
        and not pattern.match(os.path.splitext(f)[0])
    ]
    if not files:
        print("未找到任何可用的 .xlsx 文件")
        return
    dfs = []
    for f in sorted(files):
        path = os.path.join(folder_path, f)
        try:
            # 第一行是列名，第一列作为索引（合并键）
            df = pd.read_excel(path, header=0, index_col=0)
        except Exception as e:
            print(f"读取失败：{path}，原因：{e}")
            continue
        # 规范化索引（第一列），避免空白/类型不一致
        df.index = df.index.astype(str).str.strip()
        # 列名加前缀（保留原始列名，避免冲突）
        stem = os.path.splitext(f)[0]
        df.columns = [f"{stem}__{col}" for col in df.columns]
        dfs.append(df)
    if not dfs:
        print("没有成功读取的表格")
        return
    # 以索引（第一列）按“交集”对齐拼接
    merged = pd.concat(dfs, axis=1, join="inner")
    # 索引转为普通列，并命名为 "date"
    merged = merged.reset_index().rename(columns={"index": "date"})
    # 导出
    merged.to_excel(output_file, index=False, header=True)
    return merged
# data.run_data()
# factor.run_factor()
myfactor = nf.NewFactor()
myfactor.get_data('CDB').to_excel('factor/CDB.xlsx')
df_raw = merge_excels_by_first_col('factor', 'merged_CDB.xlsx')

In [1]:
import pandas as pd

df_raw = pd.read_excel('merged_CDB.xlsx')
df_raw.set_index('date', inplace=True)

# 先删除目标列为空的行
df_raw = df_raw.dropna(subset=['CDB__国开债_10Y'])

# 涨跌标签：下一期比当前期涨为 1，跌为 -1，不变为 0
df_raw['label'] = (df_raw['CDB__国开债_10Y'].shift(-1) - df_raw['CDB__国开债_10Y']).apply(
    lambda x: -1 if x > 0 else (1 if x < 0 else 0)
)

# 若为 0，则继承前一个标签
df_raw['label'] = df_raw['label'].replace(0, method='ffill')

# 删除最后一行（因为没有未来数据做标签）
df_clean = df_raw.iloc[:-1]

# 假设 df_clean 已经准备好，索引为时间或顺序
label_col = "label"
split_point = int(len(df_clean) * 0.8)

# 按顺序切分
df_train = df_clean.iloc[:split_point]
df_test  = df_clean.iloc[split_point:]

# 拆分特征和标签
X_train, y_train = df_train.drop(columns=[label_col]), df_train[label_col]
X_test,  y_test  = df_test.drop(columns=[label_col]),  df_test[label_col]

train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

C:\Users\86198\AppData\Local\Temp\ipykernel_19036\3577052761.py:15: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df_raw['label'] = df_raw['label'].replace(0, method='ffill')


In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor
predictor = TabularPredictor(label=label_col).fit(train_data,presets='best')

d:\conda\envs\pavane\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No path specified. Models will be saved in: "AutogluonModels\ag-20251020_065553"
Preset alias specified: 'best' maps to 'best_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          16
Memory Avail:       5.97 GB / 15.73 GB (37.9%)
Disk Space Avail:   31.72 GB / 274.71 GB (11.5%)
Presets specified: ['best']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1

(_ray_fit pid=26876) [1000]	valid_set's binary_error: 0.273684


(_dystack pid=37620) 	0.7248	 = Validation score   (accuracy)
(_dystack pid=37620) 	28.69s	 = Training   runtime
(_dystack pid=37620) 	0.06s	 = Validation runtime
(_dystack pid=37620) Fitting model: RandomForestGini_BAG_L2 ... Training model for up to 210.89s of the 210.82s of remaining time.
(_dystack pid=37620) 	0.664	 = Validation score   (accuracy)
(_dystack pid=37620) 	1.41s	 = Training   runtime
(_dystack pid=37620) 	0.23s	 = Validation runtime
(_dystack pid=37620) Fitting model: RandomForestEntr_BAG_L2 ... Training model for up to 209.14s of the 209.07s of remaining time.
(_dystack pid=37620) 	0.6785	 = Validation score   (accuracy)
(_dystack pid=37620) 	1.67s	 = Training   runtime
(_dystack pid=37620) 	0.28s	 = Validation runtime
(_dystack pid=37620) Fitting model: CatBoost_BAG_L2 ... Training model for up to 207.09s of the 207.01s of remaining time.
(_dystack pid=37620) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=

## 信号与重要性

In [3]:
predictor.evaluate(test_data,auxiliary_metrics=True)

{'accuracy': 0.6510538641686182,
 'balanced_accuracy': np.float64(0.6418936906557813),
 'mcc': 0.28825360223603236,
 'roc_auc': np.float64(0.7243766083947111),
 'f1': 0.6977687626774848,
 'precision': 0.669260700389105,
 'recall': 0.7288135593220338}

In [4]:
predictor.path

'd:\\mycodelife\\workshop\\yinhe\\daily_factor\\AutogluonModels\\ag-20251020_065553'

In [5]:
from autogluon.tabular import TabularPredictor
import pandas as pd

# 加载模型并预测
# predictor = TabularPredictor.load("/root/AutogluonModels/ag-20251017_002741")
predictions = predictor.predict(test_data)

# 构建结果 DataFrame
df_result = test_data.reset_index()[['date', 'CDB__国开债_10Y', 'label']].copy()
df_result['predictions'] = predictions.values

# 所有数值取负，但国债修复了所以不用
# df_result['label'] = -df_result['label']
# df_result['predictions'] = -df_result['predictions']

# 保存结果
output_file = 'prediction_results_国开债十年.xlsx'
df_result.to_excel(output_file, index=False)

print(f"预测结果已保存到 {output_file}")

预测结果已保存到 prediction_results_国开债十年.xlsx


In [6]:
import pandas as pd
from autogluon.tabular import TabularPredictor
# predictor = TabularPredictor.load("/root/AutogluonModels/ag-20251016_045333")
fi = predictor.feature_importance(data=test_data)
fi.to_excel('国开债十年.xlsx')

These features in provided data are not utilized by the predictor and will be ignored: ['20251020-因子__是否交易日', '20251020-因子交易日__是否交易日', '20251020-因子交易日__月', '20251020-因子交易日__日', '20251020-因子交易日__是否15号', '20251020-因子交易日__是否20号', '20251020-因子交易日__星期', '20251020-因子交易日__国债_10Y-国开债_10Y', '20251020-因子交易日__国债_10Y-企业债AAA+_10Y', '20251020-因子交易日__国债_10Y-中短期票据AAA+_10Y', '20251020-因子交易日__国债_10Y-国债_1Y', '20251020-因子交易日__国债_10Y-国债_3Y', '20251020-因子交易日__国债_10Y-国债_5Y', '20251020-因子交易日__国债_10Y-国债_7Y', '20251020-因子交易日__国债_30Y-国债_10Y', '20251020-因子交易日__国开债_10Y-国债_10Y', '20251020-因子交易日__国开债_5Y-国债_5Y', '20251020-因子交易日__国开债_1Y-国债_1Y', '20251020-因子交易日__中短期票据AAA+_1Y-国开债1Y', '20251020-因子交易日__中短期票据AAA+_3Y-国开债3Y', '20251020-因子交易日__中短期票据AAA+_5Y-国开债5Y', '20251020-因子交易日__中短期票据AAA+_7Y-国开债7Y', '20251020-因子交易日__中短期票据AAA+_10Y-国开债10Y', '20251020-因子交易日__中短期票据AAA+_30Y-国开债30Y', '20251020-因子交易日__国债_10Y-同业存单AAA_1Y', '20251020-因子交易日__国开债_10Y-国开债_5Y', '20251020-因子交易日__国开债_10Y-国开债_1Y', '20251020-因子交易日__R007-R001', '20251020-因子交易

# 黄金

In [2]:
import data
import new_factor as nf
import numpy as np
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
import factor
import re
import os, sys
from datetime import datetime

# 把脚本所在目录设为当前工作目录
# os.chdir(os.path.dirname(os.path.abspath(__file__)))

def merge_excels_by_first_col(folder_path, output_file="merged.xlsx"):
    # 排除：以8位日期开头、紧跟“-因子清单”的文件（不含扩展名匹配）
    pattern = re.compile(r"^\d{8}-因子清单")
    files = [
        f for f in os.listdir(folder_path)
        if f.endswith(".xlsx")
        and not f.startswith("~")
        and not pattern.match(os.path.splitext(f)[0])
    ]
    if not files:
        print("未找到任何可用的 .xlsx 文件")
        return
    dfs = []
    for f in sorted(files):
        path = os.path.join(folder_path, f)
        try:
            # 第一行是列名，第一列作为索引（合并键）
            df = pd.read_excel(path, header=0, index_col=0)
        except Exception as e:
            print(f"读取失败：{path}，原因：{e}")
            continue
        # 规范化索引（第一列），避免空白/类型不一致
        df.index = df.index.astype(str).str.strip()
        # 列名加前缀（保留原始列名，避免冲突）
        stem = os.path.splitext(f)[0]
        df.columns = [f"{stem}__{col}" for col in df.columns]
        dfs.append(df)
    if not dfs:
        print("没有成功读取的表格")
        return
    # 以索引（第一列）按“交集”对齐拼接
    merged = pd.concat(dfs, axis=1, join="inner")
    # 索引转为普通列，并命名为 "date"
    merged = merged.reset_index().rename(columns={"index": "date"})
    # 导出
    merged.to_excel(output_file, index=False, header=True)
    return merged
# data.run_data()
factor.run_factor()
myfactor = nf.NewFactor()
myfactor.get_data('gold').to_excel('factor/gold.xlsx')
df_raw = merge_excels_by_first_col('factor', 'merged_gold.xlsx')

类别: 利率利差
保存并更新数据成功
时间: 20170101 - 20251022
当前存在数据时间范围: 20170101 - 20251022
保存路径: factor/利率利差_20251022.xlsx

类别: 国债技术指标
保存并更新数据成功
时间: 20170101 - 20251022
当前存在数据时间范围: 20170101 - 20251022
保存路径: factor/国债技术指标_20251022.xlsx

类别: 国债期货技术指标
保存并更新数据成功
时间: 20170101 - 20251022
当前存在数据时间范围: 20170101 - 20251022
保存路径: factor/国债期货技术指标_20251022.xlsx

类别: 资金面
保存并更新数据成功
时间: 20170101 - 20251022
当前存在数据时间范围: 20170101 - 20251022
保存路径: factor/资金面_20251022.xlsx

类别: 国债期货价差
保存并更新数据成功
时间: 20170101 - 20251022
当前存在数据时间范围: 20170101 - 20251022
保存路径: factor/国债期货价差_20251022.xlsx

类别: 股市
保存并更新数据成功
时间: 20170101 - 20251022
当前存在数据时间范围: 20170101 - 20251022
保存路径: factor/股市_20251022.xlsx

类别: 宏观
保存并更新数据成功
时间: 20170101 - 20251022
当前存在数据时间范围: 20170101 - 20251022
保存路径: factor/宏观_20251022.xlsx

类别: 交易日期
保存并更新数据成功
时间: 20170101 - 20251022
当前存在数据时间范围: 20170101 - 20251022
保存路径: factor/交易日期_20251022.xlsx



In [1]:
import pandas as pd

df_raw = pd.read_excel('merged_gold.xlsx')
df_raw.set_index('date', inplace=True)

# 先删除目标列为空的行
df_raw = df_raw.dropna(subset=['gold__黄金收盘价'])

# 涨跌标签：下一期比当前期涨为 1，跌为 -1，不变为 0
df_raw['label'] = (df_raw['gold__黄金收盘价'].shift(-1) - df_raw['gold__黄金收盘价']).apply(
    lambda x: -1 if x > 0 else (1 if x < 0 else 0)
)

# 若为 0，则继承前一个标签
df_raw['label'] = df_raw['label'].replace(0, method='ffill')

# 删除最后一行（因为没有未来数据做标签）
df_clean = df_raw.iloc[:-1]

# 假设 df_clean 已经准备好，索引为时间或顺序
label_col = "label"
split_point = int(len(df_clean) * 0.8)

# 按顺序切分
df_train = df_clean.iloc[:split_point]
df_test  = df_clean.iloc[split_point:]

# 拆分特征和标签
X_train, y_train = df_train.drop(columns=[label_col]), df_train[label_col]
X_test,  y_test  = df_test.drop(columns=[label_col]),  df_test[label_col]

train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

C:\Users\86198\AppData\Local\Temp\ipykernel_20604\1038339937.py:15: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df_raw['label'] = df_raw['label'].replace(0, method='ffill')


In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor
predictor = TabularPredictor(label=label_col).fit(train_data,presets='best')

d:\conda\envs\pavane\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No path specified. Models will be saved in: "AutogluonModels\ag-20251022_044048"
Preset alias specified: 'best' maps to 'best_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          16
Memory Avail:       5.62 GB / 15.73 GB (35.7%)
Disk Space Avail:   33.76 GB / 274.71 GB (12.3%)
Presets specified: ['best']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1

(_ray_fit pid=13272) [1000]	valid_set's binary_error: 0.415789


(_dystack pid=5020) 	0.6708	 = Validation score   (accuracy)
(_dystack pid=5020) 	21.56s	 = Training   runtime
(_dystack pid=5020) 	0.09s	 = Validation runtime
(_dystack pid=5020) Fitting model: LightGBM_BAG_L2 ... Training model for up to 259.68s of the 259.60s of remaining time.
(_ray_fit pid=7848) 	Ran out of time, stopping training early. (Stopping on epoch 92)
(_dystack pid=5020) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=2, gpus=0, memory=0.80%)
(_dystack pid=5020) 	0.6649	 = Validation score   (accuracy)
(_dystack pid=5020) 	26.59s	 = Training   runtime
(_dystack pid=5020) 	0.07s	 = Validation runtime
(_dystack pid=5020) Fitting model: RandomForestGini_BAG_L2 ... Training model for up to 227.88s of the 227.80s of remaining time.
(_dystack pid=5020) 	0.5706	 = Validation score   (accuracy)
(_dystack pid=5020) 	1.14s	 = Training   runtime
(_dystack pid=5020) 	0.18s	 = Validation runtime
(_dystack pid=5020) Fitting mo

## 输出

In [7]:
predictor = TabularPredictor.load("d:\mycodelife\workshop\yinhe\daily_factor\AutogluonModels\\ag-20251022_044048")
predictor.evaluate(test_data,auxiliary_metrics=True)

{'accuracy': 0.5667447306791569,
 'balanced_accuracy': np.float64(0.5096477865484929),
 'mcc': 0.021985728585814913,
 'roc_auc': np.float64(0.4845358445275354),
 'f1': 0.31226765799256506,
 'precision': 0.4077669902912621,
 'recall': 0.25301204819277107}

In [8]:
from autogluon.tabular import TabularPredictor
import pandas as pd

# 加载模型并预测
# predictor = TabularPredictor.load("/root/AutogluonModels/ag-20251017_002741")
predictions = predictor.predict(test_data)

# 构建结果 DataFrame
df_result = test_data.reset_index()[['date', 'gold__黄金收盘价', 'label']].copy()
df_result['predictions'] = predictions.values

# 所有数值取负，但国债修复了所以不用
# df_result['label'] = -df_result['label']
# df_result['predictions'] = -df_result['predictions']

# 保存结果
output_file = 'prediction_results_gold.xlsx'
df_result.to_excel(output_file, index=False)

print(f"预测结果已保存到 {output_file}")

预测结果已保存到 prediction_results_gold.xlsx


In [ ]:
import pandas as pd
from autogluon.tabular import TabularPredictor
# predictor = TabularPredictor.load("/root/AutogluonModels/ag-20251016_045333")
fi = predictor.feature_importance(data=test_data)
fi.to_excel('黄金.xlsx')

These features in provided data are not utilized by the predictor and will be ignored: ['20251022-因子__是否交易日', '20251022-因子交易日__是否交易日', '20251022-因子交易日__月', '20251022-因子交易日__日', '20251022-因子交易日__是否15号', '20251022-因子交易日__是否20号', '20251022-因子交易日__星期', '20251022-因子交易日__国债_10Y-国开债_10Y', '20251022-因子交易日__国债_10Y-企业债AAA+_10Y', '20251022-因子交易日__国债_10Y-中短期票据AAA+_10Y', '20251022-因子交易日__国债_10Y-国债_1Y', '20251022-因子交易日__国债_10Y-国债_3Y', '20251022-因子交易日__国债_10Y-国债_5Y', '20251022-因子交易日__国债_10Y-国债_7Y', '20251022-因子交易日__国债_30Y-国债_10Y', '20251022-因子交易日__国开债_10Y-国债_10Y', '20251022-因子交易日__国开债_5Y-国债_5Y', '20251022-因子交易日__国开债_1Y-国债_1Y', '20251022-因子交易日__中短期票据AAA+_1Y-国开债1Y', '20251022-因子交易日__中短期票据AAA+_3Y-国开债3Y', '20251022-因子交易日__中短期票据AAA+_5Y-国开债5Y', '20251022-因子交易日__中短期票据AAA+_7Y-国开债7Y', '20251022-因子交易日__中短期票据AAA+_10Y-国开债10Y', '20251022-因子交易日__中短期票据AAA+_30Y-国开债30Y', '20251022-因子交易日__国债_10Y-同业存单AAA_1Y', '20251022-因子交易日__国开债_10Y-国开债_5Y', '20251022-因子交易日__国开债_10Y-国开债_1Y', '20251022-因子交易日__R007-R001', '20251022-因子交易

# 黄金2

In [1]:
import pandas as pd

df_raw = pd.read_excel('gold.xlsx')
df_raw.set_index('date', inplace=True)

# 先删除目标列为空的行
df_raw = df_raw.dropna(subset=['收盘价'])

# 涨跌标签：下一期比当前期涨为 1，跌为 -1，不变为 0
df_raw['label'] = (df_raw['收盘价'].shift(-1) - df_raw['收盘价']).apply(
    lambda x: -1 if x > 0 else (1 if x < 0 else 0)
)

# 若为 0，则继承前一个标签
df_raw['label'] = df_raw['label'].replace(0, method='ffill')

# 删除最后一行（因为没有未来数据做标签）
df_clean = df_raw.iloc[:-1]

# 假设 df_clean 已经准备好，索引为时间或顺序
label_col = "label"
split_point = int(len(df_clean) * 0.8)

# 按顺序切分
df_train = df_clean.iloc[:split_point]
df_test  = df_clean.iloc[split_point:]

# 拆分特征和标签
X_train, y_train = df_train.drop(columns=[label_col]), df_train[label_col]
X_test,  y_test  = df_test.drop(columns=[label_col]),  df_test[label_col]

train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

C:\Users\86198\AppData\Local\Temp\ipykernel_23468\602854208.py:15: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df_raw['label'] = df_raw['label'].replace(0, method='ffill')


In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor
predictor = TabularPredictor(label=label_col).fit(train_data,presets='best')

d:\conda\envs\pavane\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No path specified. Models will be saved in: "AutogluonModels\ag-20251022_064151"
Preset alias specified: 'best' maps to 'best_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          16
Memory Avail:       8.37 GB / 15.73 GB (53.2%)
Disk Space Avail:   33.26 GB / 274.71 GB (12.1%)
Presets specified: ['best']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1

(_ray_fit pid=7752) [1000]	valid_set's binary_error: 0.375


(_dystack pid=23208) 	0.5871	 = Validation score   (accuracy)
(_dystack pid=23208) 	4.89s	 = Training   runtime
(_dystack pid=23208) 	0.03s	 = Validation runtime
(_dystack pid=23208) Fitting model: CatBoost_r177_BAG_L1 ... Training model for up to 510.05s of the 808.26s of remaining time.
(_dystack pid=23208) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=2, gpus=0, memory=5.17%)
(_dystack pid=23208) 	0.607	 = Validation score   (accuracy)
(_dystack pid=23208) 	4.09s	 = Training   runtime
(_dystack pid=23208) 	0.0s	 = Validation runtime
(_dystack pid=23208) Fitting model: NeuralNetTorch_r79_BAG_L1 ... Training model for up to 500.68s of the 798.90s of remaining time.
(_dystack pid=23208) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=2, gpus=0, memory=0.02%)
(_ray_fit pid=21924) d:\conda\envs\pavane\Lib\site-packages\sklearn\compose\_column_transformer.py:975: Futur

(_ray_fit pid=1456) [1000]	valid_set's binary_error: 0.284483


(_dystack pid=23208) 	0.7148	 = Validation score   (accuracy)
(_dystack pid=23208) 	2.55s	 = Training   runtime
(_dystack pid=23208) 	0.06s	 = Validation runtime
(_dystack pid=23208) Fitting model: LightGBM_BAG_L2 ... Training model for up to 276.57s of the 276.43s of remaining time.
(_dystack pid=23208) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=2, gpus=0, memory=0.19%)
(_ray_fit pid=20496) 	Ran out of time, stopping training early. (Stopping on epoch 0) [repeated 7x across cluster]


...

(_dystack pid=23208) 	0.7154	 = Validation score   (accuracy)
(_dystack pid=23208) 	2.95s	 = Training   runtime
(_dystack pid=23208) 	0.06s	 = Validation runtime
(_dystack pid=23208) Fitting model: RandomForestGini_BAG_L2 ... Training model for up to 268.15s of the 268.00s of remaining time.
(_dystack pid=23208) 	0.6792	 = Validation score   (accuracy)
(_dystack pid=23208) 	0.89s	 = Training   runtime
(_dystack pid=23208) 	0.11s	 = Validation runtime
(_dystack pid=23208) Fitting model: RandomForestEntr_BAG_L2 ... Training model for up to 267.11s of the 266.97s of remaining time.
(_dystack pid=23208) 	0.676	 = Validation score   (accuracy)
(_dystack pid=23208) 	0.83s	 = Training   runtime
(_dystack pid=23208) 	0.12s	 = Validation runtime
(_dystack pid=23208) Fitting model: CatBoost_BAG_L2 ... Training model for up to 266.13s of the 265.98s of remaining time.
(_dystack pid=23208) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=2

(_ray_fit pid=20388) [1000]	valid_set's binary_error: 0.25431


(_dystack pid=23208) 	0.7013	 = Validation score   (accuracy)
(_dystack pid=23208) 	3.51s	 = Training   runtime
(_dystack pid=23208) 	0.04s	 = Validation runtime
(_dystack pid=23208) Fitting model: NeuralNetFastAI_r191_BAG_L2 ... Training model for up to 142.17s of the 142.03s of remaining time.
(_dystack pid=23208) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=2, gpus=0, memory=0.07%)
(_ray_fit pid=468) No improvement since epoch 4: early stopping
(_dystack pid=23208) 	0.683	 = Validation score   (accuracy)
(_dystack pid=23208) 	15.89s	 = Training   runtime
(_dystack pid=23208) 	0.09s	 = Validation runtime
(_dystack pid=23208) Fitting model: CatBoost_r9_BAG_L2 ... Training model for up to 121.20s of the 121.06s of remaining time.
(_dystack pid=23208) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=2, gpus=0, memory=6.37%)
(_dystack pid=23208) 	0.7078	 = Validation 

(_ray_fit pid=15576) [1000]	valid_set's binary_error: 0.25431


(_dystack pid=23208) 	0.696	 = Validation score   (accuracy)
(_dystack pid=23208) 	1.53s	 = Training   runtime
(_dystack pid=23208) 	0.05s	 = Validation runtime
(_dystack pid=23208) Fitting model: NeuralNetTorch_r22_BAG_L2 ... Training model for up to 72.31s of the 72.16s of remaining time.
(_dystack pid=23208) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=2, gpus=0, memory=0.04%)
(_ray_fit pid=23284) d:\conda\envs\pavane\Lib\site-packages\sklearn\compose\_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
(_ray_fit pid=23284)   warnings.warn(
(_dystack pid=23208) 	0.6943	 = Validation score   (accuracy)
(_dystack pid=23208) 	17.48s	 = Training   runtime
(_dystack pid=23208) 	0.08s	 = Validation runtime
(_dystack pid=23208) Fitting model: XGBoost_r33_BAG_L2 ... Training model for up

## 输出

In [3]:
predictor.evaluate(test_data,auxiliary_metrics=True)

{'accuracy': 0.6934865900383141,
 'balanced_accuracy': np.float64(0.6566647871870286),
 'mcc': 0.3432955190288033,
 'roc_auc': np.float64(0.7453863854558753),
 'f1': 0.550561797752809,
 'precision': 0.6758620689655173,
 'recall': 0.46445497630331756}

In [4]:
from autogluon.tabular import TabularPredictor
import pandas as pd

# 加载模型并预测
# predictor = TabularPredictor.load("/root/AutogluonModels/ag-20251017_002741")
predictions = predictor.predict(test_data)

# 构建结果 DataFrame
df_result = test_data.reset_index()[['date', '收盘价', 'label']].copy()
df_result['predictions'] = predictions.values

# 所有数值取负，但国债修复了所以不用
# df_result['label'] = -df_result['label']
# df_result['predictions'] = -df_result['predictions']

# 保存结果
output_file = 'prediction_results_gold2.xlsx'
df_result.to_excel(output_file, index=False)

print(f"预测结果已保存到 {output_file}")

预测结果已保存到 prediction_results_gold2.xlsx


# 绘图

In [5]:
import pandas as pd

def analyze_excel_columns(filepath):
    df = pd.read_excel(filepath)
    index_col = df.columns[0]  # 第一列，通常是日期或 ID

    results = []

    for col in df.columns[1:]:  # 跳过第一列
        col_data = df[[index_col, col]].dropna()
        if col_data.empty:
            continue

        max_val = col_data[col].max()
        min_val = col_data[col].min()
        mean_val = col_data[col].mean()

        max_index = col_data.loc[col_data[col] == max_val, index_col].values[0]
        min_index = col_data.loc[col_data[col] == min_val, index_col].values[0]

        results.append({
            '特征名（列）': col,
            '最大值': max_val,
            '最大值位置（第一列值）': max_index,
            '最小值': min_val,
            '最小值位置（第一列值）': min_index,
            '平均值': mean_val
        })

    summary_df = pd.DataFrame(results)
    print(summary_df)
    return summary_df
print(analyze_excel_columns('特征重要性.xlsx'))

                     特征名（列）       最大值                       最大值位置（第一列值）  \
0    永续五年rxtreme importance  0.007692       20251016-因子__国开债_10Y-国开债_5Y   
1                    stddev  0.015430         20251016-因子__国债期货_5Y_wr_5   
2                   p_value  0.999578       20251016-因子__国债期货_30Y_cmo_5   
3   资本债五年extreme importance  0.032628      20251016-因子__国债_30Y_收益率变动_5D   
4                  stddev.1  0.015464      20251016-因子__国债_30Y_收益率变动_5D   
5                 p_value.1  0.999696        20251016-因子__国债期货_2Y_klow2   
6           永续债七年importance  0.027885       20251016-因子__国债期货_10Y_wr_20   
7                  stddev.2  0.017467       20251016-因子__国债期货_10Y_wr_20   
8                 p_value.2  0.999777   20251016-因子__国债_5Y-FR007_IRS_5Y   
9         二级资本债七年importance  0.036254      20251016-因子__国债_30Y_收益率变动_5D   
10                 stddev.3  0.013099       20251016-因子__国债_1Y_收益率变动_5D   
11                p_value.3  0.998581                20251016-因子__是否20号   
12        FR007一年importan

In [ ]:
import pandas as pd

def top_bottom_extremes_by_category(filepath, category='importance', top_n=10):
    df = pd.read_excel(filepath)
    index_col = df.columns[0]  # 第一列用于找对应位置
    
    results = []

    # 筛选列名中包含目标关键词的列
    selected_cols = [col for col in df.columns if category.lower() in col.lower()]

    for col in selected_cols:
        col_data = df[[index_col, col]].dropna()
        if col_data.empty:
            continue

        max_val = col_data[col].max()
        min_val = col_data[col].min()
        mean_val = col_data[col].mean()

        max_index = col_data.loc[col_data[col] == max_val, index_col].values[0]
        min_index = col_data.loc[col_data[col] == min_val, index_col].values[0]

        results.append({
            '列名': col,
            '最大值': max_val,
            '最大值位置': max_index,
            '最小值': min_val,
            '最小值位置': min_index,
            '平均值': mean_val
        })

    result_df = pd.DataFrame(results)

    # 分别选出最大值前N个和最小值前N个
    top_max = result_df.nlargest(top_n, '最大值')
    bottom_min = result_df.nsmallest(top_n, '最小值')

    return top_max, bottom_min

# 示例调用方式：
# top10, bottom10 = top_bottom_extremes_by_category('特征重要性.xlsx', category='importance')
# print("Top 10 最大值:")
# print(top10)
# print("Bottom 10 最小值:")
# print(bottom10)
